## 1. 建立並保存 ONNX 模型檔案
以下是一個使用 TensorFlow 建立鳶尾花（Iris）分類模型並將其導出為 ONNX 格式的範例。該模型使用簡單的全連接層來進行分類，並轉換為 ONNX 格式，方便在 TVM 或其他 ONNX 支持的推理引擎上運行。

### 1.1 安裝必要的套件
如果尚未安裝 tensorflow 和 tf2onnx，可以使用以下命令安裝：



In [ ]:
!pip install tensorflow tf2onnx

### 1.2 建立並訓練 TensorFlow 模型
以下程式碼將建立一個簡單的神經網絡來分類鳶尾花數據集，並將其導出為 ONNX 格式。

In [19]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 載入鳶尾花資料集
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(4,)),  # 4 個特徵
    tf.keras.layers.Dense(10, activation='relu'),  # 隱藏層
    tf.keras.layers.Dense(3, activation='softmax') # 輸出層，3 個分類
])

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# 評估模型
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"模型準確率: {accuracy:.2f}")

2024-11-13 22:38:20.798397: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 22:38:20.839582: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 22:38:20.839610: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 22:38:20.839650: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 22:38:20.848293: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 22:38:20.849394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

模型準確率: 0.97


### 1.3 將模型轉換為 ONNX 格式
使用 tf2onnx 將訓練好的 TensorFlow 模型轉換為 ONNX 格式：

In [ ]:
import tf2onnx

# 將 Keras 模型轉換為 ONNX 格式
spec = (tf.TensorSpec((None, 4), tf.float32, name="float_input"),)  # 定義輸入規範
output_path = "tf_model.onnx"  # 輸出 ONNX 模型的路徑

# 轉換模型
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f"ONNX 模型已保存至 {output_path}")

In [20]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('tf_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})

# 輸出預測結果
print(pred_onnx)

[array([[5.2434858e-04, 8.7534554e-02, 9.1194111e-01]], dtype=float32)]


## 2. TVM 進行編譯產生 C
### 2.1 轉換模型為 Relay 格式
在 TVM 中，我們可以使用 ONNX 格式轉換模型，然後導入 TVM 中。如果使用者已有其他框架的模型（如 TensorFlow、Keras、PyTorch），可相應轉換為 Relay 支持的格式。

In [ ]:
!export TVM_HOME=/home/jovyan/project/ONNX-MLIR/tvm
!export PYTHONPATH=$TVM_HOME/python:$PYTHONPATH

In [ ]:
!TVM_LIBRARY_PATH=/home/jovyan/project/ONNX-MLIR/tvm/build python3 run.py

In [22]:
import tvm
print(tvm.__file__)

/home/jovyan/project/ONNX-MLIR/tvm/python/tvm/__init__.py


In [23]:
!python -c "import tvm; print(tvm.__file__)"

[22:38:39] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[22:38:39] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[22:38:39] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
/home/jovyan/project/ONNX-MLIR/tvm/python/tvm/__init__.py


In [31]:
import onnx
import tvm
from tvm import relay

# 加载 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")

# 定义输入形状（根据您的模型调整）
shape_dict = {"float_input": (1, 4)}  # 请替换为您的实际输入名和形状

# 将 ONNX 模型转换为 Relay IR
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)


In [32]:
# 设置目标为生成 C 代码
target = tvm.target.Target("c")

# 设置运行时为 'crt' 并启用 'system-lib' 选项
runtime = tvm.relay.backend.Runtime("crt", {"system-lib": True})

# 配置 AOT 执行器
executor = relay.build_module.Executor("aot", {
    "interface-api": "c",
    "unpacked-api": True,
})

# 编译模块
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params, runtime=runtime, executor=executor)


In [33]:
# 获取生成的 C 代码
c_source = lib.lib.get_source()

# 保存 C 代码到文件
with open("model.c", "w") as f:
    f.write(c_source)


In [69]:
import onnx
import tvm
from tvm import relay

# 加载 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")
shape_dict = {"float_input": (1, 4)}  # 根据您的模型输入调整形状
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)
from tvm.relay.backend import Executor, Runtime

# 设置目标为 C，并启用 system-lib
target = tvm.target.Target("c")


# 配置执行器为 AOT，接口 API 为 C，使用 unpacked API
executor = Executor("aot", {"interface-api": "c", "unpacked-api": True})

# 设置运行时为 C 运行时，并启用 system-lib
runtime = Runtime("crt", {"system-lib": True})
from tvm.contrib import utils

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params, executor=executor, runtime=runtime)
# 导出生成的 C 代码
temp = utils.tempdir()
lib.export_library(temp.relpath("deploy.tar"))

# 解压生成的代码
import tarfile
tar = tarfile.open(temp.relpath("deploy.tar"))
tar.extractall(temp.relpath("./"))

In [74]:
temp.relpath("deploy.tar")

'/tmp/tmpn17sro6c/deploy.tar'

In [76]:
!ls '/tmp/tmpn17sro6c/'

deploy.tar  devc.c  lib0.c  lib1.c


In [56]:
!tar xvf ./test/model.tar

lib1.c
devc.c
lib0.c


In [52]:
# !gcc -o model_exec main.c model.c -I../tvm/include -I../tvm/3rdparty/dlpack/include -I../tvm/3rdparty/dmlc-core/include
!gcc -O2 -o model_exec model.c main.c -I../tvm/3rdparty/dlpack/include


model.c:72:10: fatal error: tvmgen_default.h: No such file or directory
   72 | #include <tvmgen_default.h>
      |          ^~~~~~~~~~~~~~~~~~
compilation terminated.
main.c: In function ‘main’:
main.c:17:5: warning: implicit declaration of function ‘model’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
   17 |     model(input_data, output_data);
      |     ^~~~~


###  2.2 設置 microTVM 和生成 C 代碼
使用 microTVM 的 AOT（Ahead-of-Time）執行器來生成適合嵌入式設備的 C 代碼。

In [5]:
!TVM_LIBRARY_PATH=/home/jovyan/project/ONNX-MLIR/tvm/build python3 run.py

[22:01:22] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[22:01:22] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[22:01:22] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
Traceback (most recent call last):
  File "/home/jovyan/project/ONNX-MLIR/tvm-tutorial/run.py", line 23, in <module>
    lowered_module.get_lib().export_library("iris_model.c", tvm.micro.export_model_library_format)
TypeError: expor

In [43]:
target = tvm.target.Target("llvm", host="llvm")
executor = relay.build_module.Executor("aot", {
    "interface-api": "c",       # 使用 C 接口
    "unpacked-api": True        # 使用 unpacked API，减少对运行时的依赖
})
runtime = tvm.relay.backend.Runtime("crt", {"system-lib": True})  # 指定使用 C 运行时
